# 1.5. Implementing a simple Linear Regression Algorithm

## Imports

In [ ]:
import numpy as np # conda install numpy
import tensorflow as tf 
import matplotlib.pyplot as plt # conda install matplotlib
import pandas as pd # conda install pandas
import warnings
import seaborn as sb # conda install seaborn

warnings.filterwarnings('ignore')

print(tf.__version__)

## Load data and take a look at it

In [ ]:
data = pd.read_csv('airbnb new york.csv').sample(frac=1)
data.describe()

## Data preprocessing

In [ ]:
features = data[['neighbourhood_group', 'room_type', 'minimum_nights', 'number_of_reviews', 
                 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']]
#print(features.isna().sum())
features['reviews_per_month'] = features['reviews_per_month'].fillna(0)
#print(features.isna().sum())

onehot_neighborhood_group = pd.get_dummies(features['neighbourhood_group'])
onehot_room_type = pd.get_dummies(features['room_type'])
#print(onehot_room_type)

features = features.drop(columns=['neighbourhood_group', 'room_type'])
features = pd.concat([features, onehot_neighborhood_group, onehot_room_type], axis=1)
#print(features.head())

targets = data['price']

train_size = int(0.7*len(data))
X_train, X_test = features.values[:train_size, :], features.values[train_size:, :]
y_train, y_test = targets.values[:train_size], targets.values[train_size:]

print(len(X_train[0]))

## Data visualization and analysis

## The Tensorflow 2 Machine Learning Approaches

### Linear Regression

In [ ]:
class LinearModel:
    def __init__(self):
        # y_pred = W*X + b
        self.initializer = tf.keras.initializers.GlorotUniform()
    
    def loss(self, y, y_pred):
        return tf.reduce_mean(tf.abs(y - y_pred))
    
    def train(self, X, y, lr=0.00001, epochs=20, verbose=True):
            
        X = np.asarray(X, dtype=np.float32)
        y = np.asarray(y, dtype=np.float32).reshape((-1, 1)) # [1,2,3,4] -> [[1],[2],[3],[4]]
        
        self.W = tf.Variable(
            initial_value=self.initializer(shape=(len(X[0]), 1), dtype='float32'))
        self.b = tf.Variable(
            initial_value=self.initializer(shape=(1,), dtype='float32'))

        def train_step():
            with tf.GradientTape() as t:
                current_loss = self.loss(y, self.predict(X))

            dW, db = t.gradient(current_loss, [self.W, self.b])
            self.W.assign_sub(lr * dW) # W -= lr * dW
            self.b.assign_sub(lr * db)
            
            return current_loss
        
        for epoch in range(epochs):
            current_loss = train_step()
            if verbose:
                print(f'Epoch {epoch}: Loss: {current_loss.numpy()}') # <3 eager execution
                
    def predict(self, X):
        # [a, b] x [b, c]
        # X -> [n_instances, n_features] [n_features, 1]
        return tf.matmul(X, self.W) + self.b

In [ ]:
model = LinearModel()
model.train(X_train, y_train, epochs=100)


## Conclusions